<center>
<img src="../../img/ods_stickers.jpg" />
</center> 
     
## <center>  [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 

#### <center> Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko) 

# <center>Assignment #3. Spring 2019
## <center> Part 3. Gradient boosting

**In this assignment, you're asked to beat a baseline in the ["Flight delays" competition](https://www.kaggle.com/c/flight-delays-fall-2018).**

Prior to working on the assignment, you'd better check out the corresponding course material:
 1. [Classification, Decision Trees and k Nearest Neighbors](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb?flush_cache=true), the same as an interactive web-based [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn) 
 2. Ensembles:
  - [Bagging](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part1_bagging.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-1-bagging)
  - [Random Forest](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part2_random_forest.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-2-random-forest)
  - [Feature Importance](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic05_ensembles_random_forests/topic5_part3_feature_importance.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-5-ensembles-part-3-feature-importance)
 3. - [Gradient boosting](https://nbviewer.jupyter.org/github/Yorko/mlcourse_open/blob/master/jupyter_english/topic10_boosting/topic10_gradient_boosting.ipynb?flush_cache=true), the same as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/topic-10-gradient-boosting) 
   - Logistic regression, Random Forest, and LightGBM in the "Kaggle Forest Cover Type Prediction" competition: [Kernel](https://www.kaggle.com/kashnitsky/topic-10-practice-with-logit-rf-and-lightgbm) 
 4. You can also practice with demo assignments, which are simpler and already shared with solutions:
  - "Decision trees with a toy task and the UCI Adult dataset": [assignment](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution)
  - "Logistic Regression and Random Forest in the credit scoring problem": [assignment](https://www.kaggle.com/kashnitsky/assignment-5-logit-and-rf-for-credit-scoring) + [solution](https://www.kaggle.com/kashnitsky/a5-demo-logit-and-rf-for-credit-scoring-sol)
 5. There are also 7 video lectures on trees, forests, boosting and their applications: [mlcourse.ai/video](https://mlcourse.ai/video) 

### Your task is to:
 1. beat **"A3 baseline (8 credits)"** on Public LB (**0.73449** LB score)
 2. rename your [team](https://www.kaggle.com/c/flight-delays-fall-2018/team) in full accordance with the course rating
 
 This task is intended to be relatively easy. Here you are not required to upload your reproducible solution.
 
### <center> Deadline for A3: 2019 March 31, 20:59 GMT (London time)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from scipy.sparse import csr_matrix, hstack

Download data from the [competition page](https://www.kaggle.com/c/flight-delays-fall-2018/data) and change paths if needed.

In [2]:
train_df = pd.read_csv('./data/flight_delays_train.csv')
test_df = pd.read_csv('./data/flight_delays_test.csv')

In [3]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Given flight departure time, carrier's code, departure airport, destination location, and flight distance, you have to predict departure delay for more than 15 minutes. As the simplest benchmark, let's take logistic regression and two features that are easiest to take: DepTime and Distance. This will correspond to **"simple logit baseline"** on Public LB.

In [5]:
train_df['Distance'] = StandardScaler().fit_transform(train_df['Distance'].values.reshape(-1, 1))
train_df['DepTime'] = StandardScaler().fit_transform(train_df['DepTime'].values.reshape(-1, 1))

test_df['Distance'] = StandardScaler().fit_transform(test_df['Distance'].values.reshape(-1, 1))
test_df['DepTime'] = StandardScaler().fit_transform(test_df['DepTime'].values.reshape(-1, 1))

In [6]:
X_train = train_df[['Distance', 'DepTime']].values 
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df[['Distance', 'DepTime']].values

#X_train_part, X_valid, y_train_part, y_valid = \
#    train_test_split(X_train, y_train, 
#                     test_size=0.3, random_state=17)

In [7]:
#np.random.seed(17)
#shuffle_index = np.random.permutation(100000)
#X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

In [8]:
logit = LogisticRegression(C=1, random_state=17, solver='liblinear')

#### Add month feature

In [9]:
onehotencoder = OneHotEncoder(categories='auto', sparse=False)

In [10]:
train_month_encoded = onehotencoder.fit_transform((train_df['Month']).values.reshape(-1, 1))
test_month_encoded = onehotencoder.transform((test_df['Month']).values.reshape(-1, 1))

In [11]:
%%time
X_train = np.concatenate((X_train, train_month_encoded), axis=1)
X_test = np.concatenate((X_test, test_month_encoded), axis=1)

CPU times: user 8.08 ms, sys: 4.68 ms, total: 12.8 ms
Wall time: 11.7 ms


In [12]:
y_scores = cross_val_predict(logit, X_train, y_train, cv=5, method="decision_function")

In [13]:
roc_auc_score(y_train, y_scores) # 0.6873571886373713

0.6873571886373713

#### Add Unique Carrier feature

In [14]:
onehotencoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown="ignore")

In [15]:
train_carrier_encoded = onehotencoder.fit_transform((train_df['UniqueCarrier']).values.reshape(-1, 1))
test_carrier_encoded = onehotencoder.transform((test_df['UniqueCarrier']).values.reshape(-1, 1))

In [16]:
%%time
X_train = np.concatenate((X_train, train_carrier_encoded), axis=1)
X_test = np.concatenate((X_test, test_carrier_encoded), axis=1)

CPU times: user 117 ms, sys: 19.2 ms, total: 136 ms
Wall time: 31.9 ms


In [17]:
y_scores = cross_val_predict(logit, X_train, y_train, cv=5, method="decision_function")

In [18]:
roc_auc_score(y_train, y_scores) # 0.6942087894156533

0.6942087894156533

#### Add day of week feature

In [19]:
onehotencoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown="ignore")

In [20]:
train_day_of_week_encoded = onehotencoder.fit_transform((train_df['DayOfWeek']).values.reshape(-1, 1))
test_day_of_week_encoded = onehotencoder.transform((test_df['DayOfWeek']).values.reshape(-1, 1))

In [21]:
X_train = np.concatenate((X_train, train_day_of_week_encoded), axis=1)
X_test = np.concatenate((X_test, test_day_of_week_encoded), axis=1)

In [22]:
%%time
y_scores = cross_val_predict(logit, X_train, y_train, cv=5, method="decision_function")

CPU times: user 4.33 s, sys: 58.3 ms, total: 4.39 s
Wall time: 845 ms


In [23]:
roc_auc_score(y_train, y_scores) # 0.6966938356177392

0.6966938356177392

#### Add origin feature

In [24]:
onehotencoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown="ignore")

In [25]:
train_origin_encoded = onehotencoder.fit_transform((train_df['Origin']).values.reshape(-1, 1))
test_origin_encoded = onehotencoder.transform((test_df['Origin']).values.reshape(-1, 1))

In [26]:
X_train = np.concatenate((X_train, train_origin_encoded), axis=1)
X_test = np.concatenate((X_test, test_origin_encoded), axis=1)

In [27]:
%%time
y_scores = cross_val_predict(logit, X_train, y_train, cv=3, method="decision_function")

CPU times: user 3.19 s, sys: 196 ms, total: 3.38 s
Wall time: 1.3 s


In [28]:
roc_auc_score(y_train, y_scores) # 0.70071603232622

0.70071603232622

#### Add destination feature

In [29]:
onehotencoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown="ignore")

In [30]:
train_dest_encoded = onehotencoder.fit_transform((train_df['Dest']).values.reshape(-1, 1))
test_dest_encoded = onehotencoder.transform((test_df['Dest']).values.reshape(-1, 1))

In [31]:
X_train = np.concatenate((X_train, train_dest_encoded), axis=1)
X_test = np.concatenate((X_test, test_dest_encoded), axis=1)

In [32]:
%%time
y_scores = cross_val_predict(logit, X_train, y_train, cv=3, method="decision_function")

CPU times: user 3.8 s, sys: 407 ms, total: 4.21 s
Wall time: 2.1 s


In [33]:
roc_auc_score(y_train, y_scores) # 0.704392527867389

0.704392527867389

#### Add carrier/origin feature

Seems to hurt score

In [34]:
#onehotencoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown="ignore")

In [35]:
#train_carrier_origin_encoded = onehotencoder.fit_transform((train_df['Origin'] + train_df['Dest']).values.reshape(-1, 1))
#test_carrier_origin_encoded = onehotencoder.transform((test_df['Origin'] + test_df['Dest']).values.reshape(-1, 1))

In [36]:
#X_train = np.concatenate((X_train, train_carrier_origin_encoded), axis=1)
#X_test = np.concatenate((X_test, test_carrier_origin_encoded), axis=1)

In [37]:
#%%time
#y_scores = cross_val_predict(logit, X_train, y_train, cv=3, method="decision_function")

In [38]:
#roc_auc_score(y_train, y_scores)

In [39]:
#def add_origin_dest(df):
#    df['origin-destination'] = df['Origin'] + df['Dest']
#    return onehotencoder.fit_transform((df['origin-destination']).values.reshape(-1, 1)).toarray()

### Grid Search

In [40]:
params = {
    'C': np.logspace(-2, 2, 10)
    # 'penalty': ['l1','l2']
}

logit_grid_searcher = GridSearchCV(estimator=logit, param_grid=params,
                                  scoring='roc_auc', n_jobs=1, cv=5, verbose=1)

In [41]:
%%time
logit_grid_searcher.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   45.5s finished


CPU times: user 1min 36s, sys: 5.97 s, total: 1min 42s
Wall time: 45.9 s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [42]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_ # 0.7070256809581367

(0.7070256809581367, {'C': 0.027825594022071243})

In [43]:
logit_test_pred = logit_grid_searcher.predict_proba(X_test)[:, 1]

In [44]:
pd.Series(logit_test_pred, 
          name='dep_delayed_15min').to_csv('./submissions/03-logit_2feat.csv', 
                                            index_label='id', header=True)
# 0.70698

In [45]:
# TODO create a time of day - morning, afternoon, evening, late night features instead of hour

In [39]:
# TODO should shuffle training data?

#### Add origin/destination feature

Seems to hurt cv scores?

In [41]:
#def add_origin_dest(df):
#    df['origin-destination'] = df['Origin'] + df['Dest']
#    return onehotencoder.fit_transform((df['origin-destination']).values.reshape(-1, 1)).toarray()

In [42]:
#%%time
#X_train = np.concatenate((X_train, add_origin_dest(train_df)), axis=1)
#X_test = np.concatenate((X_test, add_origin_dest(test_df)), axis=1)

In [43]:
#%%time
#y_scores = cross_val_predict(logit, X_train, y_train, cv=3, method="decision_function")

In [44]:
#roc_auc_score(y_train, y_scores)

## Random Forest

In [45]:
forest_clf = RandomForestClassifier(random_state=17, n_estimators=300)

In [46]:
X_train_part, X_valid, y_train_part, y_valid = \
    train_test_split(X_train, y_train, 
                     test_size=0.3, random_state=17)

In [47]:
%%time
forest_clf.fit(X_train_part, y_train_part)
forest_valid_pred = forest_clf.predict_proba(X_valid)[:, 1]

CPU times: user 19.2 s, sys: 343 ms, total: 19.6 s
Wall time: 19.6 s


In [48]:
roc_auc_score(y_valid, forest_valid_pred) # at 100 estimators 0.6903952715035478  # at 10 estimators 0.6481071851043585

0.6511173163791452

### Grid Search

In [19]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

In [49]:
param_grid = [
    {'n_estimators': [150, 300], 'max_features': [8, 10, 12]}
    #{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

In [50]:
grid_search = GridSearchCV(forest_clf, param_grid, cv=5, scoring='roc_auc', verbose=1)

In [51]:
%%time
#grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 15.1min finished


CPU times: user 17min 36s, sys: 10.1 s, total: 17min 46s
Wall time: 15min 57s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [150, 300], 'max_features': [8, 10, 12]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [52]:
grid_search.best_score_, grid_search.best_params_ # 0.7070256809581367

(0.6746761032005117, {'max_features': 12, 'n_estimators': 300})

In [ ]:
#logit_pipe.fit(X_train, y_train)
#logit_test_pred = logit_pipe.predict_proba(X_test)[:, 1]

#pd.Series(logit_test_pred, 
#          name='dep_delayed_15min').to_csv('./submissions/02-logit_2feat.csv', 
#                                           index_label='id', header=True)

Now you have to beat **"A3 baseline (8 credits)"** on Public LB. It's not challenging at all. Go for LightGBM, maybe some other models (or ensembling) as well. Include categorical features, do some simple feature engineering as well. Good luck!